# Data Science with Python and Dask

**Author:** David-Alexandre Guenette <br />
**Date:** 2021-01-20 <br />

## Problem Definition

**Problem:** What patterns can we find in the data that are correlated with increases or decreases in the number of parking tickets issued by the New York City parking authority?

**Hypothesis:** 

 - We might find that older vehicles are more likely to receibe tickets.
 - We might find that a particular color attracts more attention from the parking authority than other colors.

## Data Gathering

#### Importing CSV files using Dask defaults

In [3]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

fy14 = dd.read_csv('./data/nyu_parking_data/parking_violations_2014.csv')
fy15 = dd.read_csv('./data/nyu_parking_data/parking_violations_2015.csv')
fy16 = dd.read_csv('./data/nyu_parking_data/parking_violations_2016.csv')
fy17 = dd.read_csv('./data/nyu_parking_data/parking_violations_2017.csv')

#### Finding the common columns between the four DataFrames

In [4]:
from functools import reduce


def set_columns(df):
    """
    Purpose : Make a set of columns from DataFrame.
    """
    
    scolumns = set(df.columns)
    return scolumns


def find_common_columns(ls):
    """
    Purpose : Takes a list of columns sets object to make a common list of columns name
    """
    
    cc = list(reduce(lambda a, i: a.intersection(i), ls))
    return cc



columns = [
    set_columns(fy14),
    set_columns(fy15),
    set_columns(fy16),
    set_columns(fy17)
]

common_columns = find_common_columns(columns)


#### Building a generic schema

In [5]:
import numpy as np
import pandas as pd

# First we need to build a dictionary that maps column names to datatypes. 
dtypes = {
 'Date First Observed': np.str,
 'Days Parking In Effect    ': np.str,
 'Double Parking Violation': np.str,
 'Feet From Curb': np.float32,
 'From Hours In Effect': np.str,
 'House Number': np.str,
 'Hydrant Violation': np.str,
 'Intersecting Street': np.str,
 'Issue Date': np.str,
 'Issuer Code': np.float32,
 'Issuer Command': np.str,
 'Issuer Precinct': np.float32,
 'Issuer Squad': np.str,
 'Issuing Agency': np.str,
 'Law Section': np.float32,
 'Meter Number': np.str,
 'No Standing or Stopping Violation': np.str,
 'Plate ID': np.str,
 'Plate Type': np.str,
 'Registration State': np.str,
 'Street Code1': np.uint32,
 'Street Code2': np.uint32,
 'Street Code3': np.uint32,
 'Street Name': np.str,
 'Sub Division': np.str,
 'Summons Number': np.uint32,
 'Time First Observed': np.str,
 'To Hours In Effect': np.str,
 'Unregistered Vehicle?': np.str,
 'Vehicle Body Type': np.str,
 'Vehicle Color': np.str,
 'Vehicle Expiration Date': np.str,
 'Vehicle Make': np.str,
 'Vehicle Year': np.float32,
 'Violation Code': np.uint16,
 'Violation County': np.str,
 'Violation Description': np.str,
 'Violation In Front Of Or Opposite': np.str,
 'Violation Legal Code': np.str,
 'Violation Location': np.str,
 'Violation Post Code': np.str,
 'Violation Precinct': np.float32,
 'Violation Time': np.str
}

## Data Cleaning and Transforming

#### Importing the NYC Parking Ticket Data

In [6]:
nyc_data_raw = dd.read_csv('./data/nyu_parking_data/*.csv', dtype=dtypes, usecols=dtypes.keys())

#### Define Class for Transforming Data

In [30]:
class Cleaner():
    
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    
    # --- Columns' Methods --- #
    
    def select_columns(self, columns):
        filtered_df = self.dataframe[columns]
        return filtered_df
    
    def drop_columns(self, columns):
        dropped_df = self.dataframe.drop(columns, axis=1)
        return dropped_df
    
    def rename_columns(self, columns):
        renamed_df = self.dataframe.rename(columns=columns)
        return renamed_df
    
    # --- Rows' Methods --- #
    
    def select_rows(self, r0, r1):
        filtered_df = self.dataframe.loc[r0:r1]
        return filtered_df
    
    
    # --- Missing Values' Methods --- #
    
    def calc_pct_missing_val(self):
        missing_values = self.dataframe.isnull().sum()
        percent_missing = ((missing_values / self.dataframe.index.size) * 100)
        return percent_missing
    
    def drop_columns_treshold(self, treshold, ls):
        columns_to_drop = list(ls[ls >= treshold].index)
        df_clean_stage01 = self.dataframe.drop(columns_to_drop, axis=1)
        return df_clean_stage01
        
    def drop_rows_treshold(self, treshold, ls):
        rows_to_drop = list(ls[(ls > 0) & (ls < treshold)].index)
        df_clean_stage02 = self.dataframe.dropna(subset=rows_to_drop)
        return df_clean_stage02
    
    def imputing_columns_missing_val(self, treshold ,ls, col_name):
        remaining_columns_to_clean = list(ls[(ls > 0) & (ls < treshold)].index)
        unknown_default_dict = dict(map(lambda columnName: (columnName, col_name), remaining_columns_to_clean))
        df_clean_stage03 = self.dataframe.fillna(unknown_default_dict)
        return df_clean_stage03

    
cleaner = Cleaner(nyc_data_raw)

#### Selecting multiple columns from a DataFrame

In [8]:
columns_to_select = ['Plate ID', 'Registration State']


with ProgressBar():
    display(cleaner.select_columns(columns_to_select).head())

[########################################] | 100% Completed |  1.4s


,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


#### Dropping multiple columns form a DataFrame

In [9]:
violationColumnNames = list(filter(lambda columnName: 'Violation' in columnName, nyc_data_raw.columns))


with ProgressBar():
    display(cleaner.drop_columns(violationColumnNames).head())

[########################################] | 100% Completed |  1.4s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,408.0,F1,BBBBBBB,ALL,ALL,GY,0,2013.0,-,0.0
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,408.0,C,BBBBBBB,ALL,ALL,WH,0,2012.0,-,0.0
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,408.0,F7,BBBBBBB,ALL,ALL,NaN,0,0.0,-,0.0
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,408.0,F1,BBBBBBB,ALL,ALL,WH,0,2010.0,-,0.0
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,408.0,E1,BBBBBBB,ALL,ALL,BR,0,2012.0,-,0.0


#### Renaming columns from a DataFrame

In [10]:
new_columns = {
    
    'Plate ID': 'License Plate'
}


nyc_data_renamed = cleaner.rename_columns(new_columns)

#### Getting multiple rows by index

In [11]:
with ProgressBar():
    display(cleaner.select_rows(100, 200).head())

[########################################] | 100% Completed |  1.4s


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
100,1294727205,XBD7628,VA,PAS,08/04/2013,17,SUBN,JEEP,P,14510,...,GRY,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
101,1294727461,R613159,IL,PAS,07/17/2013,17,SDN,VOLKS,P,14510,...,BLUE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
102,1294727473,6TCX735,CA,PAS,07/18/2013,17,SDN,MAZDA,P,14510,...,BLACK,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
103,1294727497,ZWZ43K,NJ,PAS,08/10/2013,17,SUBN,LINCO,P,14510,...,NaN,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
104,1295357240,T624858C,NY,PAS,07/22/2013,21,SUBN,TOYOT,X,28790,...,SILVE,0,2012.0,-,0.0,NaN,NaN,NaN,NaN,NaN


#### Counting missing values in a DataFrame

In [12]:
with ProgressBar():
    percent_missing = cleaner.calc_pct_missing_val().compute()
percent_missing

[########################################] | 100% Completed |  3min 33.8s


Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

#### Dropping columns that have more than 50% missing values

In [13]:
nyc_data_clean_stage1 = cleaner.drop_columns_treshold(50, percent_missing)

#### Dropping rows with missing data

In [20]:
cleaner = Cleaner(nyc_data_clean_stage1)

nyc_data_clean_stage2 = cleaner.drop_rows_treshold(5, percent_missing)

#### Imputing Multiple Columns With Missing Values

In [28]:
cleaner = Cleaner(nyc_data_clean_stage2)

nyc_data_clean_stage3 = cleaner.imputing_columns_missing_val(50, percent_missing, 'Unknown')


In [29]:
# Checking the success of the filling/dropping operations

with ProgressBar():
    print(nyc_data_clean_stage3.isnull().sum().compute())
nyc_data_clean_stage3.persist()

[########################################] | 100% Completed |  4min 50.4s
Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                  

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Date First Observed,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
npartitions=142,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,object,object,object,object,uint16,object,object,object,uint32,uint32,uint32,object,object,float32,float32,float32,object,object,object,object,object,object,object,object,float32,object,object,object,object,object,float32,float32,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
